In [76]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import os as _dir

csv_filelist = ["Mouse_metadata.csv","Study_results.csv"]
working_dir = _dir.getcwd()
for csv in csv_filelist:   
    for root, dirs, files in _dir.walk(working_dir):
        if csv in files:
            if csv == csv_filelist[0]:
                mouse_metadata_path = (_dir.path.join(root, csv))
            else:
               study_results_path =  (_dir.path.join(root, csv))
# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
combine_resultset = pd.merge(study_results, mouse_metadata, on='Mouse ID')
# Display the data table for preview
combine_resultset.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [73]:
 # Checking the number of mice.
combine_resultset['Mouse ID'].describe()
combine_resultset['Mouse ID'].value_counts().head()

g989    13
j296    10
o562    10
k210    10
t198    10
Name: Mouse ID, dtype: int64

In [74]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
combine_resultset.loc[combine_resultset.duplicated(subset=['Mouse ID','Timepoint'])== True]

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
861,g989,0,45.000000,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [75]:
combine_resultset = combine_resultset.set_index('Mouse ID').drop(labels='g989').reset_index()
combine_resultset['Mouse ID'].value_counts().head()

j296    10
k403    10
x402    10
k210    10
i557    10
Name: Mouse ID, dtype: int64